# Matplotlib and Seaborn

### Handy hints 

Some Jupyter shortcut keys:

- Esc : get into Command mode (leaves Edit mode)
- Enter : edit a cell (puts you in Edit mode)
- h : see help (see all commands)
- Shift+Enter or Ctrl+Enter : run the code in the cell
- a / b : add new cell above/below
- m : turn current cell into a Markdown cell
- y : turn current cell into a Code cell
- 1-6 : turn current cell into a heading (a type of Markdown cell)

You can get help on Python objects and functions with `help()` or the `?` operator.

In general, we are using plotting libraries that return objects encapsulating the plot. You can check the type of these returned objects with `type()`. 

Some of the plotting libraries we use need to communicate a lot of data to the browser. Current versions of Jupyter shouldn't have a problem, but if you are running an older version, you may need to launch this notebook with a higher data rate limit: `jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000`

## Setup 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# This causes Jupyter to display any matplotlib plots directly in the notebook
# It also works for seaborn, since seaborn uses matplotlib to render plots
%matplotlib inline
from matplotlib import pyplot as plt

Be aware that Seaborn automatically changes Matplotlib's defaults *on import*. Not only your Seaborn plots, but also your Matplotlib plots, will look different once Seaborn is imported. If you don't want this behaviour, you can call `sns.reset_orig()` after import, or `import seaborn.apionly as sns` in the first place.

In [ ]:
import seaborn as sns

## Data 

For this workshop we're going to load in two main datasets:

### LTEE data 

This data is from the [Long Term Evolution Experiment (LTEE)](http://myxo.css.msu.edu/ecoli/). This experiment has been running for over 30 years and over 50,000 E. coli generations, and is still ongoing. Twelve separate populations of E. coli have been propagated for the life of the experiment. Every 500 generations, each population is cloned and stored, allowing researchers to study evolutionary behaviour over the long term, and to "rewind and replay" alternate evolutionary trajectories by propagating from an earlier clone. 

Several interesting events have occurred during the experiment. Some populations have spontaneously developed hypermutator phenotypes. In addition, around generation 31,000 one population, Ara-3, spontaneously developed a rare and novel Cit+ mutation, giving it the ability to metabolise citrate in the substrate.

There have been many publications from this experiment. A handful:

- [Blount et al 2008: Historical contingency and the evolution of a key innovation in an experimental population of Escherichia coli](https://www.pnas.org/content/105/23/7899) - on the spontaneous development of citrate metabolisation and on potentiating mutations
- [Tenaillon et al 2016: Tempo and mode of genome evolution in a 50,000-generation experiment](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4988878) - various investigations by sequencing and variant-calling over 50,000 generations of clones, including discussion of hypermutator phenotypes and genetic drift vs natural selection.

Sequence data from clones is available, but for this workshop we'll just be using some published tabular data.

A version of this dataset is also used by the [Data Carpentry lessons on Genomics](https://datacarpentry.org/genomics-workshop/).

In this lesson we'll use a large flat file containing both sample metadata on each clone, and information on observed mutations in their genomes.

In [ ]:
# If you have done the pandas and data tidying workshop and saved the final file,
# you can try reading that file in here if you prefer
ltee = pd.read_csv('ltee_merged.csv',
                   index_col = 'Strain ID')

### House sales data 

This data represents house sales in Seattle in 2014 and 2015. We have information on the houses themselves: location, size, quality, view, and whether the house is tagged as 'waterfront' or not. We also have information on the date and price of each sale.

In [ ]:
sales = pd.read_csv("housing-data-10000.csv", 
                    usecols=['id','date','price','lat','long', 'zipcode',
                             'waterfront','view','grade','sqft_living'],
                    parse_dates=['date'], 
                    dtype={'zipcode': 'category',
                           'waterfront': 'bool'})

# We'll downsample in order to make plots a bit simpler and smaller to store
# In particular Altair defaults to a limit of 5000 observations
sales = sales.sample(3000)

### Demo data 

We'll read in the classic `iris` and `mtcars` datasets for demo purposes.

In [ ]:
iris = pd.read_csv('iris.csv')
cars = pd.read_csv('mtcars.csv')

## Matplotlib 

Matplotlib is the oldest and still the fundamental plotting library in Python. It has a huge range of capabilities. Many other libraries (including Seaborn) use Matplotlib as a back-end renderer.

Today we're focussing on plotting tabular data. We won't touch on all Matplotlib's capabilities. If you want to see more of the range of things Matplotlib can do, you can look through the [Matplotlib gallery](https://matplotlib.org/gallery.html.), or try out this excellent [Matplotlib tutorial](https://www.labri.fr/perso/nrougier/teaching/matplotlib/).

An example Matplotlib plot with legend and annotation:

In [ ]:
x = [1,2,3,4,5]
y = [2,5,10,17,26]
y2 = [1,4,9,11,9]

fig, ax = plt.subplots()
ax.plot(x, y, c='blue', label='Projected')
ax.scatter(x, y2, c='red', label='Actual')
fig.legend()
ax.annotate("where it all went wrong", 
                                 xy=(3,10), xytext=(1,12),
                                 arrowprops={'width':2})

fig.savefig('example_matplotilb.png')

## Seaborn 

Seaborn builds on Matplotlib. Some nice features are:

- works directly with Pandas dataframes, concise syntax
- lots of plot types, including some more advanced options
- statistical plotting: many plots do summary statistics for you
- good default aesthetics and easy control of aesthetics
- uses Matplotlib, so can use all Matplotlib backends (incl lots of image file formats)
- underlying Matplotlib objects can be tweaked directly

For completeness, here's the plot we made before:

In [ ]:
df = pd.DataFrame({
    'Time': [1,2,3,4,5],
    'Projected': [2,5,10,17,26],
    'Actual': [1,4,9,11,9]
})

fig, ax = plt.subplots()
sns.scatterplot(data=df, x='Time', y='Actual', color='red', ax=ax)
sns.lineplot(data=df, x='Time', y='Projected', color='blue', ax=ax)

ax.set_ylabel('Huge profits')

ax.annotate("where it all went wrong", 
                                 xy=(3,10), xytext=(1,12),
                                 arrowprops={'width':2})

Notice that we can add changes like annotations in exactly the same way, as we have Matplotlib Figure and Axes objects.

### Seaborn and Pandas 

In some cases we can use Seaborn by passing in lists (or arrays or series) directly:

In [ ]:
sns.barplot(x=['A','B','C'], y=[33,44,20])

However Seaborn is aware of Pandas and it is very common to use Seaborn directly with DataFrames. Plotting functions can take a DataFrame as their `data` parameter and then refer directly to column names:

In [ ]:
sns.barplot(data=iris, x='species', y='petal_length')

Here Seaborn has interpreted the `x` and `y` arguments as field names in the supplied DataFrame. Notice also that Seaborn has performed the summary statistics for us - in this case, using the default `estimator`, which is `mean()`. 

Notice also what happens if we simply swap the `x` and `y` parameters. Seaborn will automatically deduce that the categorical or string-like variable must be the bar labels, and the numeric variable must be the numeric axis:

In [ ]:
sns.barplot(data=iris, y='species', x='petal_length')

What happens if the grouping is not obviously categorical? How could we fix this?

In [ ]:
sns.barplot(data=cars, x='acceleration', y='origin')

#### Exercises: 

**1:** Create a count plot using `sns.countplot()` on the `ltee` data, showing how many clones have each `Mutator` phenotype. Note that you do not need to specify the `y` axis variable for a countplot, just the `x` axis variable (i.e. category).

**2:** Create a (vertical) bar plot using the `sales` data, showing how house prices vary with the value of the property `grade`.

Bar plots are often deplored as a way of showing statistical estimates, as only the top of the bar is really important, and the bar itself is a visual distraction. A point plot is an alternative, and plots like box plots can show more information. Several other plot types also show distributional information within categories.

**3:** Reproduce the plot you just made, using instead each of the Seaborn functions:

- pointplot()
- boxplot()
- violinplot()  (try the `scale` parameter)
- boxenplot()
- stripplot() [SEE WARNING]  (try the `jitter` parameter)
- swarmplot() [SEE WARNING]

Note what sort of information about the distribution is shown by each.

WARNING: `stripplot()` and `swarmplot()` will plot individual data points. There are too many house sales to easily display in this way - you may want to subsample the dataframe with e.g.  `data=sales.sample(100)`.

### Hue 

Many Seaborn plotting functions take a `hue` parameter. This colours the plot elements by some categorical variable, but more than this, summary statistics are calculated for each level of the hue variable.

In [ ]:
sns.scatterplot(data=ltee, 
                x='Generation', 
                y='Synonymous Base Substitutions', 
                palette='bright') # hue='Population')

In [ ]:
sns.lineplot(data=ltee, 
            x='Generation', 
            y='Synonymous Base Substitutions', 
            palette='bright') # hue='Population')

In [ ]:
sns.lmplot(data=sales, x='sqft_living', y='price') #, hue='waterfront')

#### Exercises:

1. Subset the `ltee` dataset to get only the clones where the `Mutator` value is "PM"
2. Create an `lmplot` of `Total Mutations` over time (i.e. against `Generation`). Do this without a `hue` parameter, then add in `Population` as the `hue` parameter. 
3. Try adding the `hue` parameter to one of your previous plots of some other type - for instance, a box plot.

If we'd wanted to colour scatter points by some continuous variable, `hue` can be made to work, but doesn't really make sense as it is intended for discrete values. In this specific case, we could pass our colour variable down to the underlying Matplotlib scatter call via the `scatter_kws` parameter. We'll look more at this later. Or, if we're not trying to fit a linear model, we could just use Seaborn aesthetics around a Matplot scatter plot:

In [ ]:
sample = sales.sample(200)

with sns.axes_style('white'):
    fig, ax = plt.subplots(figsize=(8,8))
    ax.scatter(x=sales['long'], y=sales['lat'], c=sales['price'].apply(np.log), 
               alpha=0.5, s=7, cmap='Reds')

For discrete color palettes, as needed by the `hue` parameter, Seaborn has a `color_palette()` function to generate a useful range of palettes. You can find [a tutorial on choosing color palettes here](https://seaborn.pydata.org/tutorial/color_palettes.html). 

### Compound plots 

Seaborn has some plotting functions which create more complex figures made of multiple subplots. These include `cat()`, `catplot()`, `jointplot()`, `lmplot()` and `clustermap()`. Let's see a few examples:

In [ ]:
# jointplot shows a scatter or density plot, with marginal distributions
sns.jointplot(data=sales, x='sqft_living', y='price') #, kind='reg')

In [ ]:
# pairplot shows pairwise relationships between variables
sns.pairplot(data=ltee,
             vars=['Generation', 'Total Mutations', 'Nonsynonymous Base Substitutions', 'IS Element Insertions'])
             #hue='Mutator')

In [ ]:
# catplot conditions different subplots on different categorical variable values
# we map variables to row and column of a grid of plots (as well as to hue)
# in this example, we just use columns, and so have only one row
sns.catplot(data=cars, kind='box',
               x='year', y='mpg', 
               row='origin',
               aspect=2)

#### Exercise:

Create a joint plot using the `ltee` data showing `Cell size` against `Relative fitness`

#### Exercise: 

Design a plot using `sns.catplot()`, to show the relationship between house price and: grade, view, and whether the property is waterfront. Available channels of information are:

- x and y coordinates
- hue
- row and column of subplot (`row` and `col`)

You can set the `kind` parameter to the kind of plot you want to make: point, bar, count, box, violin, or stripplot.

You do not have to use all of these channels - in fact your plot may be difficult to take in if you do.

### Colour and Palettes

Seaborn has good colour options. There are a few ways we could want to use access colours:

* Specify an individual colour for some plot element. Matplotlib named colours can be used, or rgb values specified. Also check out the `sns.xkcd_rgb` dictionary of 954 named colours from the XKCD colour survey - for instance, `sns.xkcd['fire engine red']` is a colour.
* Specify a colormap, for mapping a continuous value to colour. All Matplotlib colormaps can be used by name. You can see these under the `plt.cm` module. Seaborn's `light_palette()` and `dark_palette()` functions can also generate custom colourmaps easily.
* Specify a discrete colour palette (a list of colours), for mapping a discrete or categorical variable to colour. In Seaborn, there is a distinction between colour palettes and colormaps. In general, you can create a colour palette by explicitly listing some colours, or by selecting a series of colours along some colormap. There are several functions, including `color_palette()`, `light_palette()`, `dark_palette()`, `diverging_palette()` and `xkcd_palette()`, which can produce many discrete colour palettes of whatever size you need. 

In [ ]:
# An example discrete colour palette of 7 colours, based on the XKCD colour "denim blue"
# palplot is a function to visualise a palette
palette = sns.light_palette("denim blue", n_colors=7, input='xkcd')
sns.palplot(palette)

In [ ]:
# Equivalently (to illustrate that we can use an rgb value directly)
denim_blue = sns.xkcd_rgb["denim blue"]
print(denim_blue)
palette = sns.light_palette(denim_blue, n_colors=7)
print(palette)
sns.palplot(palette)

**Exercise:** Try out the Seaborn function `choose_diverging_palette()` in your notebook (it requires no arguments). You can assign the result to a variable.

Let's try a heatmap. Unlike most Seaborn functions, heatmaps take data in wide form. We'll need to pivot our long-form data to get a table of numbers, indexed by two variables. The heatmap function will then transform each number to a colour via a continuous colourmap.

#### Exercise: 

Use `ltee.pivot_table()` to produce a table showing the (average) number of mutations per Population and Generation. The x-axis (column headers) should be the values of the `Generation` variable, and the rows (index) should be the values of the `Population` variable. If you're new to pivot tables in Pandas, check the example below that uses the `cars` dataset.

In [ ]:
# Here's an example using the toy dataset
# Avoid looking at the details first if you want to solve the above without hints!

speed_table = cars.pivot_table(index='origin', columns='year', values='mpg', aggfunc=np.mean)
speed_table

In [ ]:
sns.heatmap(speed_table, vmin=0, annot=True)

#### Exercises:

* If you haven't already, produce a heatmap with the `ltee` data as described above. You will probably want to leave off the annotations unless your plot is very large. 
* Specify a different colourmap using the `cmap` parameter to `heatmap`. An ascending (not diverging) colourmap is appropriate for counts that are all positive.
* Some populations have far more mutations than others, and so our heatmap doesn't really show detail for the lower end of the scale. Try to plot a heatmap where colour is based on the *log* of the mutation count.  

### Seaborn and Matplotlib 

Seaborn plotting functions call Matplotlib plotting functions, and we can pass arguments down to these underlying functions.

For instance, `lmplot()` calls `scatter()` and `plot()` to draw points and lines. We can pass arguments down using `scatter_kws` and `line_kws`:

In [ ]:
sns.lmplot(data=sales, x='grade', y='price',
           scatter_kws={'alpha': 0.3},
           line_kws={'linestyle': 'dashed', 'color': 'red'})

**Exercise:** Which underlying plotting function is called by your `catplot` above? You can try passing arguments to it with catplot's `kwargs`.

Let's look at the object returned by a Seaborn plot function.

In [ ]:
g = sns.countplot(ltee['Mutator'])

In [ ]:
type(g)

This is a Matplotlib `Axes` object. We can use all the usual `Axes` methods to tweak the plot. What's more, if we have an existing `Axes` object, we can ask Seaborn to draw the plot onto it by passing it in as the `ax` parameter:

In [ ]:
fig, myaxes = plt.subplots(figsize=(7,5), facecolor=sns.xkcd_rgb['pale pink'])

sns.countplot(ltee['Mutator'], color='purple', ax=myaxes)

And the plot is still attached to `fig`:

In [ ]:
fig

The simpler Seaborn plotting functions return `Axes` objects, and can take an `Axes` as a parameter. More complex functions like `jointplot()` and `catplot()` need to make multiple subplots. What do they return?

In [ ]:
g = sns.pairplot(data=ltee,
             vars=['Generation', 'Total Mutations', 'Nonsynonymous Base Substitutions', 'IS Element Insertions'],
             hue='Mutator')

In [ ]:
type(g)

In [ ]:
type(g.fig)

In [ ]:
g.axes

In [ ]:
g.fig

As we can see, these more complex functions produce entire Matplotlib `Figure` objects, which can contain multiple `Axes`. The `Figure`, however, comes wrapped in a Seaborn class, which provides some convenient functions to manipulate the figure properties.

In [ ]:
# g.set sets a property on all Axes in the Figure
# set x-axis to log scale:
g.set(xscale='log')
g.fig

The Seaborn classes commonly returned are:

- JointGrid : a central bivariate plot with two marginal univariate plots. Used by `jointplot()`. 
- PairGrid : a grid of subplots for plotting pairwise relationships. Has convenience functions for mapping plots onto diagonal and non-diagonal elements. Used by `pairplot()`.
- FacetGrid : a grid of subplots showing the same relationship, conditioned on some variable across different subplots. Designed to map fields of a Pandas DataFrame to different rows, columns, and hues. Used by `catplot()` and `lmplot()`.

It's possible to instantiate these classes yourself for custom plots.